In [2]:
from code_tokenizer import *

code = """  
if(a > 10)
{
    a = 10
}
"""

print(color_by_char_type(code))
parsed = parse(code)
print(parsed)

  
if(a > 10)
{
    a = 10
}

['if', [['operation', '>', [['word', 'a'], ['number', '10']]], ['operation', '=', [['word', 'a'], ['number', '10']]]]]
